In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model


In [2]:
# Veri artırma
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)


In [3]:
train_generator = datagen.flow_from_directory(
    'input/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 4747 images belonging to 4 classes.


In [4]:
val_generator = datagen.flow_from_directory(
    'input/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 1185 images belonging to 4 classes.


In [5]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [6]:
for layer in base_model.layers:
    layer.trainable = False


In [7]:
# Özelleştirilmiş sınıflandırma bloğu
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

In [8]:
# Modeli oluştur
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Eğit
model.fit(train_generator,validation_data=val_generator,epochs=3 )

C:\Users\BU\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
149/149 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.7111 - loss: 4.9008 - val_accuracy: 0.9435 - val_loss: 0.1480
Epoch 2/3
149/149 ━━━━━━━━━━━━━━━━━━━━ 301s 2s/step - accuracy: 0.9237 - loss: 0.1993 - val_accuracy: 0.9654 - val_loss: 0.1024
Epoch 3/3
149/149 ━━━━━━━━━━━━━━━━━━━━ 299s 2s/step - accuracy: 0.9296 - loss: 0.2107 - val_accuracy: 0.9747 - val_loss: 0.0614


In [10]:
loss, accuracy = model.evaluate(val_generator)
print("Validation Accuracy:", accuracy)


38/38 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.9748 - loss: 0.0633
Validation Accuracy: 0.9789029359817505
